In [ ]:
!pip install datasets
!pip install "accelerate>=0.29.0"
!pip install evaluate
!pip install optuna

In [ ]:
import sys
import numpy as np
import pickle
import re
from nltk.stem import SnowballStemmer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
nltk.download("stopwords")
nltk.download('wordnet')
tqdm.pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# excel_file = "data.mail.xlsx"
# all_sheets = pd.read_excel(excel_file, sheet_name=None)
# sheets = all_sheets.keys()
# for sheet_name in sheets:
#     sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
#     sheet.to_csv("%s.csv" % sheet_name, index=False)

In [ ]:
# stop = set(stopwords.words("russian"))
# stemmer = SnowballStemmer("russian")
# tokenizer = RegexpTokenizer(r'\w+')

# def clear_text(text):
#     return ' '.join(stemmer.stem(x) for x in tokenizer.tokenize(text.lower())[:256] if x not in stop)
#     return text

In [ ]:
# df = pd.read_csv("deti.mail.csv")
# df

In [ ]:
# # df["text"] = df["Article Text"].progress_apply(clear_text)
# df["text"] = df["Article Text"]

In [ ]:
# df["labels"], indexer = df["Keywords"].factorize()

In [ ]:
# df["labels"].value_counts()

In [ ]:
df = pd.read_csv("final_set.csv")

In [ ]:
mapper = {
    "Наука": "Наука и технологии",
    "Технологии": "Наука и технологии",
    "Гаджеты": "Наука и технологии",

    "Политика": "Общество",
    "Законы": "Общество",

    "Кино": "Кино и телевидение",
    "Телевидение": "Кино и телевидение",

    "Соцсети": "Люди"
}
df["category"] = df["category"].progress_apply(lambda x: mapper.get(x, x))

100%|██████████| 61666/61666 [00:00<00:00, 713319.13it/s]


In [ ]:
df = df[ (df.category != "Опросы")]

In [ ]:
cnt = df["category"].value_counts()
df = df[df["category"].apply(lambda x: cnt[x]) > 100]

In [ ]:
df = df.dropna()

In [ ]:
df["labels"], indexer = df["category"].factorize()
df["labels"].value_counts()

labels
12    23176
2     22049
10     6745
9      6095
0      1248
3       552
5       372
4       360
1       288
11      208
6       120
7       120
8       120
Name: count, dtype: int64

In [ ]:
g = df.groupby('labels')
new_df = g.apply(lambda x: x.sample(100).reset_index(drop=True))

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
model_name = "DeepPavlov/rubert-base-cased-sentence"
tokenizer = tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

In [ ]:
text_train_x, text_test_x, train_y, test_y = train_test_split(new_df["text"].values, new_df["labels"].values, test_size=0.2, random_state=0)

In [ ]:
from datasets import Dataset


train = Dataset.from_dict({"text": text_train_x, "label": train_y})
test = Dataset.from_dict({"text": text_test_x, "label": test_y})

In [ ]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

Map:   0%|          | 0/260 [00:00<?, ? examples/s]

In [ ]:
def init_model(trial):
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(indexer)).to('cuda')

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# training_args = TrainingArguments(
#     output_dir="results",
#     learning_rate=2e-5,
#     per_device_train_batch_size=64,
#     per_device_eval_batch_size=64,
#     num_train_epochs=20,
#     weight_decay=0.01,
# )

training_args = TrainingArguments(
    output_dir="results",
    overwrite_output_dir = 'True',
    evaluation_strategy="steps",
    eval_steps=20,
    logging_steps = 40,
    learning_rate = 5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    seed=0,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=None,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    model_init=init_model,
)


trainer.train()



# trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'cla

Step,Training Loss,Validation Loss
20,No log,2.063599
40,2.031600,1.487285
60,2.031600,1.124900
80,1.140200,0.951277
100,1.140200,0.796653
120,0.785600,0.655551
140,0.785600,0.724793
160,0.491800,0.530976
180,0.491800,0.504561
200,0.304200,0.471090


TrainOutput(global_step=650, training_loss=0.36325662275919546, metrics={'train_runtime': 236.1917, 'train_samples_per_second': 22.016, 'train_steps_per_second': 2.752, 'total_flos': 342078153830400.0, 'train_loss': 0.36325662275919546, 'epoch': 5.0})

In [ ]:
pred = trainer.predict(tokenized_test).predictions.argmax(axis=1)

In [ ]:
f1_score(test_y, pred, average="micro")

0.8923076923076924

###Test

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(LogisticRegression())),
])

In [ ]:
text_clf.fit(text_train_x, train_y)

In [ ]:
import tensorflow as tf

In [ ]:
pred = text_clf.predict(text_test_x)

In [ ]:
f1_score(test_y, pred, average="micro")